In [1]:
!pip install geomstats

    100% |████████████████████████████████| 122kB 2.1MB/s ta 0:00:01
    100% |████████████████████████████████| 6.1MB 770kB/s ta 0:00:01
    100% |████████████████████████████████| 829kB 755kB/s ta 0:00:01
  Running setup.py bdist_wheel for autograd ... done
  Stored in directory: /Users/tarrysingh/Library/Caches/pip/wheels/72/6f/c2/40f130cca2c91f31d354bf72de282922479c09ce0b7853c4c5
  Running setup.py bdist_wheel for future ... done
  Stored in directory: /Users/tarrysingh/Library/Caches/pip/wheels/0c/61/d2/d6b7317325828fbb39ee6ad559dbe4664d0896da4721bf379e
Successfully built autograd future
  Found existing installation: h5py 2.7.1
    Uninstalling h5py-2.7.1:
      Successfully uninstalled h5py-2.7.1


In [2]:
"""Trains a simple convnet on the MNIST dataset.
Convolutional Layers are trained on the Hypersphere.
Based on the keras mnist cnn example.
"""
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import numpy as np
from geomstats.hypersphere import Hypersphere

Using TensorFlow backend.
Using numpy backend


In [3]:
batch_size = 128
num_classes = 10
epochs = 2

In [4]:
# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [6]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [9]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
    
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()

hypersphere_dimension = 17
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape,
                 kernel_manifold=Hypersphere(hypersphere_dimension)))
model.add(Conv2D(64, kernel_size=(3, 3),
                 activation='relu',
                 kernel_manifold=Hypersphere(hypersphere_dimension)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.1),
              metrics=['accuracy'])

conv_weights_cnn = model.get_weights()[2]
norms_before_training = np.linalg.norm(
        conv_weights_cnn.reshape(hypersphere_dimension+1, -1), axis=0)
print(model.fit(x_train, y_train,
                batch_size=batch_size,
                epochs=epochs,
                verbose=1,
                validation_data=(x_test, y_test)))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Make sure the norms of the weights are the same before and after training
# meaning that we stayed on the hypersphere.
conv_weights_cnn = model.get_weights()[2]
norms_after_training = np.linalg.norm(
        conv_weights_cnn.reshape(hypersphere_dimension+1, -1), axis=0)
np.testing.assert_array_almost_equal(norms_before_training,
        norms_after_training, decimal=2)

TypeError: ('Keyword argument not understood:', 'kernel_manifold')